In [ ]:
%matplotlib inline
figsize = (10,3)

# Input Datasets  

In [ ]:
import pandas, numpy
pandas.options.display.float_format = '{:,.4f}'.format
data_folder = '../data'
date_format='%Y-%m-%d' #truncate datetimes to dates
id_columns = ["id","company_id","invoice_id","account_id","customer_id"]
id_column_types = dict(zip(id_columns,[str] * len(id_columns)))
invoices = pandas.read_csv(data_folder + '/invoice.csv', na_values='inf', dtype=id_column_types,
                           parse_dates=['invoice_date', 'due_date', 'cleared_date'], date_format=date_format)
payments = pandas.read_csv(data_folder + '/invoice_payments.csv', na_values='inf', dtype=id_column_types,
                           parse_dates=['transaction_date'], date_format=date_format)

## Dataset Definitions & Relationships

We have two input datasets: invoices and their payments.
- Payments are amounts in time, which are directly mapped to companies. 
- Invoices can have multiple payments, but usually only have 1. 

In [ ]:
invoices.dtypes

In [ ]:
payments.dtypes

In [ ]:
#The join key will be invoice_id, so it must be unique (and it is).
invoices.id.value_counts(dropna=False).value_counts(dropna=False)\
.to_frame(name="ids").rename_axis('invoices_per_id')

In [ ]:
#all payments are represented in both datasets 
len(set(payments.invoice_id) - set(invoices.id))

In [ ]:
#7% of invoices do not have payments yet
len(set(invoices.id) - set(payments.invoice_id))/invoices.__len__()

In [ ]:
#invoices usually have one payment but may have more
payments.invoice_id.value_counts(dropna=False).value_counts(dropna=False, normalize=True)\
.to_frame(name="invoices").rename_axis('payments_per_invoice')

## Entity Definitions & Relationships

- Company: business entity for which Tesorio is forecasting cash collected. There are only two. Each company collects using multiple currencies from multiple customers. 
- Account: **In this limited dataset, accounts and companies are synonymous, so we ignore accounts.**  
- Customer: metadata about an invoice which is specific to each company. Each customer can have multiple currencies.

In [ ]:
invoices.groupby("company_id")[["customer_id","currency"]].nunique()

In [ ]:
invoices.groupby("customer_id").company_id.nunique().value_counts()\
.to_frame(name='customers').rename_axis('companies_per_customer')

In [ ]:
invoices.groupby(["customer_id"]).currency.nunique().value_counts()\
.to_frame(name='customers').rename_axis('currencies_per_customer')

In [ ]:
invoices.groupby("company_id").account_id.nunique().to_frame(name="unique_accounts")

In [ ]:
invoices.groupby("account_id").company_id.nunique().value_counts()\
.to_frame(name='count').rename_axis('companies_per_account')

## Data Cleaning Needs

### Payments

Transaction data begins in 2011 and ends 2021-05-18. We will assume this is when the data was pulled. 

In [ ]:
payments.__len__()

In [ ]:
payment_stats = payments.describe(include='all')
payment_stats.loc['% populated'] = payment_stats.loc['count']/payments.__len__()
payment_stats

In [ ]:
last_transaction_date = payments.transaction_date.max()
first_transaction_date = payments.transaction_date.min()
first_transaction_date, last_transaction_date

In [ ]:
#converted_amount is reliable
(((payments.amount * payments.root_exchange_rate_value) - payments.converted_amount).abs()).max()

In [ ]:
payments[payments.amount.isnull()!=payments.converted_amount.isnull()].__len__()

In [ ]:
payments.select_dtypes(include='float').hist(bins=50, figsize=figsize, layout=(1,3))

### Invoices

Must become active within the date range of the transactions data to ensure completeness.

In [ ]:
#opened outside of payment data time period or after they were due - need to filter 
(invoices.loc[invoices.invoice_date>last_transaction_date].__len__(), 
invoices.loc[invoices.invoice_date<first_transaction_date].__len__(), 
invoices.loc[invoices.invoice_date.dt.to_period('M')>invoices.due_date.dt.to_period('M')].__len__())

In [ ]:
invoices = invoices.loc[(invoices.invoice_date>=first_transaction_date) &
                        (invoices.invoice_date<=last_transaction_date) & 
                        (invoices.invoice_date.dt.to_period('M')<=invoices.due_date.dt.to_period('M'))]

In [ ]:
#to compare to payments. Are we holding the customer accountable to USD or their own currency?
invoices['converted_amount_inv'] = invoices.amount_inv * invoices.root_exchange_rate_value

In [ ]:
invoices_stats = invoices.describe(include='all')
invoices_stats.loc['% populated'] = invoices_stats.loc['count']/invoices.__len__()
invoices_stats

In [ ]:
invoices['months_allowed'] = invoices.due_date.dt.to_period('M') - invoices.invoice_date.dt.to_period('M')
invoices.months_allowed = invoices.months_allowed.map(lambda m: m.n if not pandas.isnull(m) else None)
#almost all invoices are due immediately or within 3 months. filter out the rest
invoices.months_allowed.value_counts(normalize=True, dropna=False)

In [ ]:
invoices['months_billing'] = invoices.cleared_date.dt.to_period('M') - invoices.invoice_date.dt.to_period('M')
invoices.months_billing = invoices.months_billing.map(lambda m: m.n if not pandas.isnull(m) else None)
#almost all invoices are cleared within a year. filter out ones that cleared before they opened. will clip to 12.  
invoices.months_billing.value_counts(normalize=True, dropna=False)

### Exchange Rate

Exchange rates vary for both payments and open invoices. Customers would expect to pay the amount they were originally invoiced in their own currency, not the USD amount originally invoiced. Therefore, we should use raw amounts to determine how much is paid vs due. 

In [ ]:
# USD is not is always 1 - it varies a lot
currency_ranges = invoices.groupby("currency").root_exchange_rate_value.describe(percentiles=[])
(currency_ranges['max']/currency_ranges['min']).sort_values().plot(kind='bar', title="Exchange Rate Spread Ratio")

In [ ]:
# 1.6% of USD invoices have an exchange rate unequal to 1
invoices_usd = invoices.query("currency=='USD'").copy()
invoices_usd['exchange_rate_is_1'] = invoices_usd['root_exchange_rate_value'] == 1
1 - invoices_usd.exchange_rate_is_1.mean()

In [ ]:
# USD exchange rate variations from 1 tend to be invoices which took longer to clear
# This suggests that the invoice exchange rate is "current state data." 
time_to_clear = invoices_usd.cleared_date - invoices_usd.invoice_date
invoices_usd['months_to_clear'] = time_to_clear.map(lambda t: round(t.days/30))

In [ ]:
invoices_usd.groupby("exchange_rate_is_1").months_to_clear.agg(['mean','count'])

In [ ]:
invoices_usd.groupby(invoices_usd.months_to_clear.clip(upper=13, lower=-1))\
.exchange_rate_is_1.mean().plot(title='% of USD Invoices With Exchange Rate Equal to 1', figsize=figsize)

### Invoice status vs cleared date

All invoices have a date cleared. 
When an invoice is open, the date cleared is set to the future, and seems to be an assumed value. 

In [ ]:
invoices.loc[invoices.cleared_date.isnull()].__len__()

In [ ]:
invoices.status.value_counts(normalize=True, dropna=False).to_frame(name="% of Invoices")

In [ ]:
invoices.loc[invoices.cleared_date.isnull() != (invoices.status == 'OPEN'),['status','cleared_date']]\
.value_counts(dropna=False)

In [ ]:
#all open invoices have the same cleared date, which is in the future relative to the latest transaction
invoices.loc[invoices.status == 'OPEN'].cleared_date.value_counts(dropna=False)

In [ ]:
#all cleared invoices have a cleared date within the payments data window
invoices.loc[invoices.status == 'CLEARED', ['invoice_date','cleared_date']].agg(['min','max'])

In [ ]:
#open invoices are already active
invoices.loc[invoices.status == 'OPEN', ['invoice_date','due_date']].agg(['min','max'])

### Merging & Checking for Consistency

- No individual payments are more than their invoices. 
- Exchange rates vary across payments.
- Companies are consistent between payments and invoices, when payments are present. 
- Amounts make the most sense in their original currencies vs in USD

In [ ]:
payments['transaction_month'] = payments.transaction_date.dt.to_period('M')
invoice_payments = invoices.rename(columns={"id":"invoice_id","amount_inv":"amount",
                                            "converted_amount_inv":"converted_amount"})\
.merge(payments, on="invoice_id", how='left', suffixes=('_inv', '_pmt'))

In [ ]:
invoice_payments.invoice_id.nunique()

In [ ]:
duplicated_columns = [col.replace('_pmt','') for col in invoice_payments.columns if col.endswith('_pmt')]
for col in  duplicated_columns:
    inconsistent_rows = invoice_payments.loc[invoice_payments[col + '_pmt']!=invoice_payments[col + '_inv']]
    print(f"{col}: {inconsistent_rows.__len__()/invoice_payments.__len__()} inconsistent rows in merged dataset")

In [ ]:
invoice_payments.query("company_id_pmt!=company_id_inv").company_id_pmt.value_counts(dropna=False)

In [ ]:
invoice_payments.query("amount_pmt!=amount_inv")[['amount_pmt','amount_inv']].describe()

In [ ]:
#no payment is more than the invoice amount in the original currency
invoice_payments.loc[invoice_payments.amount_pmt>invoice_payments.amount_inv].__len__()

In [ ]:
#converting to USD creates payments that are higher than invoice totals
invoice_payments.loc[invoice_payments.converted_amount_pmt>invoice_payments.converted_amount_inv].__len__()

### Business Questions for the Data

In [ ]:
invoice_payments['amount_pmt_pct'] = (invoice_payments.amount_pmt/invoice_payments.amount_inv)

In [ ]:
# Rougly 12% of payments are partial
(invoice_payments.amount_pmt_pct.dropna()<1).mean()

In [ ]:
invoice_payments.amount_pmt_pct\
.plot(kind="hist",bins=50, title="% of Invoice Collected with Payment", figsize=figsize)

In [ ]:
invoice_payments.sort_values(by=['invoice_id','transaction_date'], inplace=True)

In [ ]:
invoice_payments.groupby("invoice_id").amount_pmt_pct.cumsum()\
.plot(kind="hist",bins=50, title="Summed % of Invoice Collected with Payment", figsize=figsize)

In [ ]:
#small percent of payments represent overpayments
invoice_payments['pmt_pct_cum'] = invoice_payments.groupby("invoice_id").amount_pmt_pct.cumsum()
(invoice_payments.pmt_pct_cum>1).mean()

In [ ]:
#invoices with no transactions: use payments data end date as date of 0 amount 
invoice_payments.transaction_month = invoice_payments.transaction_month\
.fillna(last_transaction_date.to_period('M'))

In [ ]:
#there can be multiple transactions per month
invoice_payments.groupby(["invoice_id","transaction_month"]).transaction_date\
.count().value_counts(normalize=True).head()

In [ ]:
#the transactions are not duplicates
invoice_payments.groupby(["invoice_id","transaction_month"]).amount_pmt\
.nunique().value_counts(normalize=True).head()

In [ ]:
#but they are almost always on the same day
invoice_payments.groupby(["invoice_id","transaction_month"]).transaction_date.nunique()\
.value_counts(normalize=True).head()

In [ ]:
payment_totals = invoices.set_index('id')
payment_totals['pmt_pct_cum'] = invoice_payments.groupby("invoice_id").pmt_pct_cum.max().fillna(0)
payment_totals['transaction_date_max'] = invoice_payments.groupby("invoice_id").transaction_date.max()
payment_totals['collected_date'] = invoice_payments.query("pmt_pct_cum>=1")\
.groupby("invoice_id").transaction_date.min()

#### Comparing invoice status and % collected 

- Invoices with cleared status can still have amounts remaining. 
- Invoices with open status are rarely collected. 

In [ ]:
# define invoice as collected if payments meet invoice amount in original currencies. 91% are collected
payment_totals['collected'] = payment_totals.collected_date.isnull()==False
payment_totals.collected.mean()

In [ ]:
#define cleared based on status. 97% are cleared
payment_totals['cleared'] = payment_totals.status=='CLEARED'
payment_totals.cleared.mean()

In [ ]:
#6% of invoices have a mismatch between collected and cleared 
(payment_totals.collected!=payment_totals.cleared).mean()

In [ ]:
#on average, 94% of cleared invoices are fully collected, compared to <1% of open ones
#cleared invoices have 96% of their amounts collected on average
payment_totals.groupby("status", as_index=False)[['collected','pmt_pct_cum']].mean()

In [ ]:
#67% of invoices that have not been collected are cleared nonetheless 
payment_totals.groupby("collected", as_index=False).cleared.mean()

#### Cleared Invoices

Cleared invoices may or may not be collected. If not collected, cleared invoices tend to be more overdue, suggesting that invoices must be cleared at some point.

In [ ]:
cleared_invoices = payment_totals.query("cleared == True").copy()
cleared_invoices['months_late'] = \
(cleared_invoices.collected_date.fillna(cleared_invoices.transaction_date_max).dt.to_period('M')\
- cleared_invoices.due_date.dt.to_period('M')).map(lambda m: m.n if not pandas.isnull(m) else None)

In [ ]:
cleared_invoices.groupby("collected").months_late.agg(['mean','min','max'])

#### Comparing date cleared to date collected

Rarely, there can be a delay between the date an invoice is collected to when it is cleared. We will only forecast invoices when they are open AND not collected. 

In [ ]:
payment_totals['clear_delay_months'] = (payment_totals.cleared_date.dt.to_period('M') \
- payment_totals.collected_date.dt.to_period('M')).map(lambda m: m.n if not pandas.isnull(m) else None)

In [ ]:
payment_totals.__len__()

In [ ]:
payment_totals.clear_delay_months.describe(percentiles=[0.001,0.999])

# Structuring Data for Business Problem

- The model will handle OPEN invoices and classify how many months in the future they will be collected. 
- Define an invoice as open between its invoice date and date cleared or collected, whichever is first. 

## Creating transaction periods to model historical invoices

To model the data, we have to look the invoices in each prior period they were open and calculate when they are collected relative to that time. 

To ensure completeness, the periods we use for modeling must fall within the date range of the transactions data. Since we will forecast a year in advance, the forecast period must also be at a year prior to when the transactions data ends. 

In [ ]:
import numpy

def forecast_periods(invoice_date, last_billing_date):
    period_start = max(invoice_date,first_transaction_date.to_period('M'))
    period_end = min(last_billing_date,(last_transaction_date - pandas.DateOffset(years=1)).to_period('M'))
    return pandas.period_range(period_start, period_end)

payment_totals['last_forecast_date'] = payment_totals[['cleared_date','collected_date']].min(axis=1)
payment_totals['forecast_month'] = numpy.vectorize(forecast_periods)\
(payment_totals.invoice_date.dt.to_period('M'), payment_totals.last_forecast_date.dt.to_period('M'))
invoice_forecast_periods = payment_totals.reset_index().explode('forecast_month').dropna(subset=['forecast_month'])
invoice_forecast_periods.forecast_month.agg(['min','max'])

## Live test cases: current open invoices

Invoices that we will predict after creating the model, without knowing the accuracy of the predictions. 
Per the instructions, we only predict collection dates for open invoices. 

Use the day after the payments data ends as the present date. 

In [ ]:
present_date = last_transaction_date + pandas.DateOffset(days=1)
open_invoices = invoices.query("status=='OPEN'").copy()
open_invoices['forecast_month'] = present_date.to_period('M')

In [ ]:
open_invoices.forecast_month.agg(['min','max','count'])

## Process inputs for model training and predictions

Months have lower kurtosis than periods. 

In [ ]:
def process_model_inputs(invoices_at_time_periods):
    raw_input_columns = ['id','invoice_date', 'months_allowed','amount_inv', 'converted_amount_inv',
                         'currency','company_id','customer_id','forecast_month']
    output_col = 'collected_date'
    if output_col in invoices_at_time_periods.columns:
        raw_input_columns += [output_col, 'cleared_date']
    output_df = invoices_at_time_periods[raw_input_columns]
    #remove months_allowed > 3
    output_df = output_df[output_df.months_allowed.between(0,3)]
    #only forecast when the invoice is active. 
    output_df = output_df[output_df.forecast_month>=output_df.invoice_date.dt.to_period('M')]
    output_df['months_billing'] = (output_df.forecast_month \
                                  - output_df.invoice_date.dt.to_period('M')).map(lambda m: m.n).clip(upper=12)
    output_df['months_late'] = output_df.months_billing - output_df.months_allowed
    output_df.forecast_month = output_df.forecast_month.dt.to_timestamp()
    return output_df

open_invoices_to_score = process_model_inputs(open_invoices)
invoices_periods_to_model = process_model_inputs(invoice_forecast_periods)

In [ ]:
open_invoices_to_score.describe(include='all', percentiles=[0.001,0.999]).T

## Selecting prediction target

Predict months til collected relative to forecast date. 

Normalization:
- If the invoice isn't collected within the payments data time period, assume it's collected the day after, which we are using as the present date. 
- Clip collection period to 13 months, which is outside the forecast window.

In [ ]:
invoices_periods_to_model['months_til_collected'] = \
(invoices_periods_to_model.collected_date.dt.to_period('M') \
- invoices_periods_to_model.forecast_month.dt.to_period('M')).map(lambda m: m.n if not pandas.isnull(m) else None)
# why we clip outliers 
invoices_periods_to_model.months_til_collected.value_counts(normalize=True, dropna=False).head(20)

In [ ]:
#normalized values
invoices_periods_to_model['months_til_collected_norm'] = invoices_periods_to_model.months_til_collected.fillna(
    (present_date.to_period('M') - invoices_periods_to_model.forecast_month.dt.to_period('M')).map(lambda m: m.n)
).clip(upper=13)
invoices_periods_to_model.months_til_collected_norm.value_counts(normalize=True, dropna=False)

In [ ]:
invoices_periods_to_model.months_til_collected_norm.plot(kind='hist', bins=14, figsize=figsize, layout=(1,3), 
title="Months Til Collected, Up to 1 Year (collections 13+ months in the future are outside the forecast window)")

In [ ]:
invoices_periods_to_model.months_til_collected_norm.kurtosis()

In [ ]:
invoices_periods_to_model['periods_til_collected'] = \
(invoices_periods_to_model.months_til_collected_norm/(invoices_periods_to_model.months_allowed+1)).clip(upper=13)

invoices_periods_to_model.periods_til_collected.plot(kind='hist', bins=14, figsize=figsize, layout=(1,3), 
      title="Billing Periods Til Collected (collections 13+ months in the future are outside the forecast window)")

In [ ]:
invoices_periods_to_model.periods_til_collected.kurtosis()

In [ ]:
invoices_periods_to_model.drop(columns=['collected_date','cleared_date','periods_til_collected'], 
                               inplace=True, errors='ignore')
invoices_periods_to_model.describe(include='all', percentiles=[0.001,0.999]).T

## Analyze Input Data

In [ ]:
invoices_to_model = invoices_periods_to_model.query("months_billing==0").copy()\
.rename(columns={"forecast_month":"invoice_month"})
invoices_to_model['uncollected'] = invoices_to_model.months_til_collected.isnull()

### By Dates

In [ ]:
invoices_periods_to_model.groupby("forecast_month").id.count()\
.plot(kind='area', title="Invoices by Forecast Month", figsize=figsize)

In [ ]:
invoices_to_model.groupby("invoice_month").id.count()\
.plot(kind='area', title="Invoices by Invoice Month", figsize=figsize)

### By Currency

Some currencies have very low collection rates. This may be due to currency fluctuations.

In [ ]:
invoices_to_model.groupby("currency")\
.agg({"months_til_collected":["mean","std"],"id":"count","uncollected":"mean"})\
.sort_values(by=('uncollected','mean'), ascending=False)

### Trends Over Time

Invoice collection time and inability to collect have been trending down, which are good signs for Tesorio's business. 

In [ ]:
invoices_to_model.groupby("invoice_month").months_til_collected.mean()\
.plot(kind='line', title="Average Months to Collect by Invoice Month", figsize=figsize)

In [ ]:
invoices_to_model.groupby("invoice_month").uncollected.mean()\
.plot(kind='line', title="% Invoices Uncollected by Invoice Month", figsize=figsize)

### By Customer 

We have trouble collecting from some customers, regardless of their currency. 

In [ ]:
customer_averages = invoices_to_model.set_index("customer_id").select_dtypes(include=['float','int','boolean'])\
.reset_index().groupby("customer_id").mean()
customer_averages.hist(bins=50, figsize=(10,5), layout=(2,4))

In [ ]:
customer_stats = invoices_to_model.groupby("customer_id").uncollected.agg(['count','mean'])\
.add_prefix('uncollected_').sort_values(by="uncollected_mean", ascending=False)
customer_stats.query("uncollected_count>=30").uncollected_mean\
.plot(kind='hist', figsize=figsize, title="Customers with 30+ Invoices: % Uncollected", bins=50)

In [ ]:
western_customer_stats = invoices_to_model.query("currency in ('USD','EUR','GBP')")\
.groupby("customer_id").uncollected.agg(['count','mean'])\
.add_prefix('uncollected_').sort_values(by="uncollected_mean", ascending=False)
western_customer_stats.query("uncollected_count>=30").uncollected_mean\
.plot(kind='hist', figsize=figsize, title="US and European Customers with 30+ Invoices: % Uncollected", bins=50)

# Business Analysis

## Business Motivation

Cash collections don't follow due dates. On average:

- 6% of total cash due each month is unpaid, equating to a \\$471K average deficit.
- Total cash collected each month is 9% off from the amount due, equating to a $571K average difference in cash flow. 

In [ ]:
amount_due = invoices\
.groupby(invoices.due_date.dt.to_period('M')).converted_amount_inv.sum().to_frame(name="amount_due_usd")
amount_paid = payments.rename(columns={"transaction_month":"due_month"})\
.groupby("due_month").converted_amount.sum().to_frame(name="amount_paid_usd")
business_motivation = amount_due.join(amount_paid, how='inner').reset_index(names='due_date')\
.query(f"due_date>'{first_transaction_date}' and due_date<'{last_transaction_date}'")
business_motivation['pct_unpaid'] = 1 - (business_motivation.amount_paid_usd/business_motivation.amount_due_usd)
business_motivation['unpaid'] = business_motivation.amount_due_usd - business_motivation.amount_paid_usd
business_motivation.set_index('due_date', inplace=True)

In [ ]:
business_motivation.pct_unpaid.plot(figsize=figsize, title="% Unpaid (USD Due)")

In [ ]:
business_motivation.pct_unpaid.mean(), business_motivation.pct_unpaid.abs().mean()

In [ ]:
business_motivation.unpaid.plot(figsize=figsize, title="USD Unpaid")

In [ ]:
business_motivation.unpaid.mean(), business_motivation.unpaid.abs().mean()

## Data Science Benchmark

Define & Quantify: customers' mean absolute % error each period from USD due.

In [ ]:
invoices_to_model['due_month'] = invoices_to_model.invoice_month.dt.to_period('M') \
+ invoices_to_model.months_allowed

In [ ]:
amount_due = invoices_to_model.groupby(["company_id","due_month"]).converted_amount_inv.sum()\
.to_frame(name="amount_due_usd")
amount_paid = payments.rename(columns={"transaction_month":"due_month","invoice_id":"id"})\
.merge(invoices_to_model[["id"]], on="id",how="inner").groupby(["company_id","due_month"]).converted_amount.sum()\
.to_frame(name="amount_paid_usd")
benchmark = amount_due.join(amount_paid, how='left').reset_index()\
.query(f"due_month>'{first_transaction_date}' and due_month<'{last_transaction_date}'")
benchmark['pct_unpaid'] = 1 - (benchmark.amount_paid_usd/benchmark.amount_due_usd)
benchmark['abs_pct_error'] = benchmark.pct_unpaid.abs()

In [ ]:
benchmark.groupby("company_id")[['pct_unpaid','abs_pct_error']].mean()

In [ ]:
benchmark[['pct_unpaid','abs_pct_error']].mean()

## 